In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [18]:
def scrape_league_cap():
    #Create list of leagues
    leagues = ['nba', 'nfl', 'mlb']
    #Create empty list
    df_list = []
    for i in leagues:
        url = 'https://www.spotrac.com/' + i + '/cba/'
        #Scrape the data
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        table = soup.find('table')
        headers = [th.text.strip() for th in table.find_all('th')]
        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])
        #Create dataframe
        df = pd.DataFrame(data, columns=headers)
        df_list.append(df)

    #Split df_list into individual dataframes
    nba_df = df_list[0]
    nfl_df = df_list[1]
    mlb_df = df_list[2]
    return nba_df, nfl_df, mlb_df

In [97]:
def clean_dataframes(input_df, league):
    df = input_df.copy()
    #Set year column as index
    df.set_index('Year', inplace=True)
    
    #Split the Cap columns

    if league != 'MLB':
        df[['Highest_Cap_Team', 'Cap_Max']] = df['Highest Team Cap'].str.split('\n', expand=True)
        df[['Lowest_Cap_Team', 'Cap_Min']] = df['Lowest Team Cap'].str.split('\n', expand=True)
        df[['Highest_Paid_Player', 'Highest_Player_Salary']] = df['Highest Player Cap'].str.split('\n', expand=True)
        #Drop the original columns that were split
        df.drop(['Highest Team Cap', 'Lowest Team Cap', 'Highest Player Cap'], axis=1, inplace=True)
        #Rename the 'Cap Maximum' Column
        df.rename(columns={'Cap Maximum': 'Salary_Cap'}, inplace=True)
        #Change the Cap Change column names
        df.rename(columns={'Cap $ +/-': 'Cap_Delta_Dollar', 'Cap % +/-': 'Cap_Delta_Percent',}, inplace=True)
    else:
        df[['Highest_Cap_Team', 'Cap_Max']] = df['Highest Team Payroll'].str.split('\n', expand=True)
        df[['Lowest_Cap_Team', 'Cap_Min']] = df['Lowest Team Payroll'].str.split('\n', expand=True)
        df[['Highest_Paid_Player', 'Highest_Player_Salary']] = df['Highest Player Payroll'].str.split('\n', expand=True)
        #Drop the original columns that were split
        df.drop(['Highest Team Payroll', 'Lowest Team Payroll', 'Highest Player Payroll'], axis=1, inplace=True)
        #Rename the 'Cap Maximum' Column
        df.rename(columns={'Comp Balanace Tax': 'Comp_Balance'}, inplace=True)
        #Change the Cap Change column names
        df.rename(columns={'Tax $ +/-': 'Cap_Delta_Dollar', 'Tax % +/-': 'Cap_Delta_Percent',}, inplace=True)

    #Change the Cap Change column names
    df.rename(columns={'Cap $ +/-': 'Cap_Delta_Dollar', 'Cap % +/-': 'Cap_Delta_Percent',}, inplace=True)

    
    #Remove the dollar sign and commas from all columns
    df = df.replace({'\$': '', ',': ''}, regex=True)
    #Remove the percent sign from the Cap_Delta_Percent column
    df['Cap_Delta_Percent'] = df['Cap_Delta_Percent'].str.replace('%', '')
    #Convert all columns to numeric except 'Highest_Paid_Player', 'Highest_Cap_Team', 'Lowest_Cap_Team'
    df = df.apply(pd.to_numeric, errors='ignore')
    df['Cap_Delta_Percent'] = df['Cap_Delta_Percent']/100

    #Add string of the league as a prefix to each column
    df.columns = [league + '_' + col for col in df.columns]
    #Make all the column names lowercase
    df.columns = df.columns.str.lower()
    return df
    

In [98]:
nba_df, nfl_df, mlb_df = scrape_league_cap()
nba_df = clean_dataframes(nba_df, 'NBA')
mlb_df = clean_dataframes(mlb_df, 'MLB')
nfl_df = clean_dataframes(nfl_df, 'NFL')

In [140]:
#Merge the nba and mlb dataframes
df = pd.merge(nba_df, mlb_df, left_index=True, right_index=True, how='inner')
#Merge to the nfl dataframe
df = df.merge(nfl_df, how='left')
df.to_csv('output/SalaryCap_Full.csv')

In [141]:
nba_df.to_csv('output/NBA_SalaryCap.csv')
mlb_df.to_csv('output/MLB_SalaryCap.csv')
nfl_df.to_csv('output/NFL_SalaryCap.csv')